In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
from os.path import join, isdir
from pathlib import Path
import datetime
import pytz

import pandas as pd
import geopandas as gpd
import numpy as np

from covidcaremap.data import (processed_data_path, 
                               external_data_path,
                               published_data_path,
                               local_data_path)

In [ ]:
ccm_csv_df = pd.read_csv(
    published_data_path('us_healthcare_capacity-facility-CovidCareMap.csv'), 
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object
          }
)

In [ ]:
ccm_csv_df.head()

In [ ]:
now_utc = datetime.datetime.now()
now_est = now_utc.astimezone(pytz.timezone("US/Eastern"))

current_time = now_est.strftime("%Y-%m-%d-%H%M%S")
current_date = now_est.strftime("%Y-%m-%d")
print(current_date, current_time)

In [ ]:
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/ICUBedsHospital.csv > {processed_data_path('florida/raw/')}/ICUBedsHospital_{current_date}.csv 
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/HospitalBedsHospital.csv > {processed_data_path('florida/raw/')}/HospitalBedsHospital_{current_date}.csv 
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/COVIDHospitalizationsCounty.csv > {processed_data_path('florida/raw/')}/COVIDHospitalizationsCounty_{current_date}.csv 

In [ ]:
fl_icu_raw = pd.read_csv(processed_data_path(f'florida/raw/ICUBedsHospital_{current_date}.csv'), encoding='utf-8', sep=',')
fl_hosp_raw = pd.read_csv(processed_data_path(f'florida/raw/HospitalBedsHospital_{current_date}.csv'), encoding='utf-8', sep=',')

In [ ]:
fl_icu_raw.head()

In [ ]:
fl_icu_raw = pd.pivot_table(fl_icu_raw, 
                        values='Measure Values',
                        index=['County','FileNumber','ProviderName'], 
                        columns=['Measure Names'], aggfunc=np.sum).sort_values(['County','FileNumber']).reset_index()

fl_hosp_raw = pd.pivot_table(fl_hosp_raw, 
                        values='Measure Values',
                        index=['County','FileNumber','ProviderName'], 
                        columns=['Measure Names'], aggfunc=np.sum).sort_values(['County','FileNumber']).reset_index()

In [ ]:
icu_all_idx = fl_icu_raw[fl_icu_raw['County']=='All'].index
fl_icu_raw = pd.concat([fl_icu_raw.iloc[icu_all_idx],fl_icu_raw.drop(icu_all_idx, axis=0)], axis=0).reset_index(drop=True)

hosp_all_idx = fl_hosp_raw[fl_hosp_raw['County']=='All'].index
fl_hosp_raw = pd.concat([fl_hosp_raw.iloc[hosp_all_idx],fl_hosp_raw.drop(hosp_all_idx, axis=0)], axis=0).reset_index(drop=True)

In [ ]:
fl_icu_raw.head()

In [ ]:
fl_icu_raw.to_csv(processed_data_path(f'florida/ICU_BedsHospital1_crosstab_{current_date}.csv'), index=False)
fl_hosp_raw.to_csv(processed_data_path(f'florida/Hospital_BedsHospital1_crosstab_{current_date}.csv'), index=False)

In [ ]:
update_date = sorted([f.stem for f in Path(processed_data_path('florida/')).iterdir() if 'csv' in f.suffix], reverse=True)[0].split('_')[-1]
update_date

In [ ]:
fl_hosp = pd.read_csv(processed_data_path(f'florida/Hospital_BedsHospital1_crosstab_{update_date}.csv'), encoding='utf-8', sep=',')
fl_icu = pd.read_csv(processed_data_path(f'florida/ICU_BedsHospital1_crosstab_{update_date}.csv'), encoding='utf-8', sep=',')

In [ ]:
fl_icu.shape, fl_hosp.shape

In [ ]:
fl_icu.head()

In [ ]:
fl_hosp.drop(0,inplace=True), fl_icu.drop(0,inplace=True)

In [ ]:
fl_hosp['Total Staffed Bed Capacity'] = fl_hosp['Total Staffed Bed Capacity'].apply(lambda x: int(x.replace(',','')))
fl_icu['Total AdultICU Capacity'] = fl_icu['Total AdultICU Capacity'].apply(lambda x: int(x.replace(',','')))
fl_hosp['Bed Census'] = fl_hosp['Bed Census'].apply(lambda x: int(x.replace(',','')))
fl_icu['Adult ICU Census'] = fl_icu['Adult ICU Census'].apply(lambda x: int(x.replace(',','')))

In [ ]:
fl_hosp.head()

In [ ]:
fl_icu.head()

In [ ]:
facility_match_df = pd.read_csv(processed_data_path('fl_facilitymatching_2020-08-14.csv'))

In [ ]:
facility_match_df.tail()

In [ ]:
facility_matched_dict = {k:v for k,v in facility_match_df[['ProviderName','Matched CCM_ID']].values}

In [ ]:
facility_matched_dict

In [ ]:
manual_override_data = pd.read_csv(
    external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'),
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object}
)

In [ ]:
manual_override_cols = list(manual_override_data.columns)
manual_override_cols

In [ ]:
ccm_csv_df.columns

In [ ]:
manual_override_cols

In [ ]:
col_replace = {
    'DH-OBJECTID': 'DH-ID',
    'HCRIS-Provider Number': 'HCRIS-ID',
    'HIFLD-ID': 'HIFLD-ID'
}

manual_override_cols = [col_replace.get(n, n) for n in manual_override_cols]

In [ ]:
manual_override_cols

In [ ]:
override_dict = {
    'Staffed All Beds - SOURCE': f"FL AHCA {update_date}",
    'Staffed ICU Beds - SOURCE': f"FL AHCA {update_date}",
    'All Bed Occupancy Rate - SOURCE': f"FL AHCA {update_date}",
    'ICU Bed Occupancy Rate - SOURCE': f"FL AHCA {update_date}",
    "Manual Override Reason": f"More recent data from {update_date}",
    "Manual Override New Data Source": "FL AHCA"
}
list(override_dict.keys())

In [ ]:
fl_hosp['ProviderName']

In [ ]:
manual_override_cols

In [ ]:
facility_matched_dict

In [ ]:
facility_matched_dict

In [ ]:
fl_icu.dtypes

In [ ]:
manual_override_data.head()

In [ ]:
ccm_fl = ccm_csv_df[ccm_csv_df['State'] == 'FL']

In [ ]:
ccm_fl.head()

In [ ]:
ccm_fl['Manual Override Reason'] = 'More recent source'
ccm_fl['Manual Override New Data Source'] = 'FL AHCA'
ccm_fl.loc[:,'Staffed All Beds'] = np.nan
ccm_fl.loc[:,'Staffed ICU Beds'] = np.nan
ccm_fl.loc[:,'All Bed Occupancy Rate'] = np.nan
ccm_fl.loc[:,'ICU Bed Occupancy Rate'] = np.nan

In [ ]:
ccm_fl.T

In [ ]:
fl_hosp[fl_hosp['ProviderName'] == 'UF HEALTH LEESBURG HOSPITAL']

In [ ]:
l = []
not_added = []
for p in fl_hosp['ProviderName']:
    
    try: ccm_id = str(facility_matched_dict[p])
    except Exception as exc: 
        print('not found: ',exc)
        ccm_id = ''
    found_ccm_record = ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]
    #if ccm_id == 'FL-CCM-0001': print(p, found_ccm_record)
    
    # ignore adding new facilities for now, only update existing matched facilities
    if len(ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]) > 0:
        
        total_staffed_beds = fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        icu_staffed_beds = fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        
        if total_staffed_beds > 0:
            hosp_occupancy = fl_hosp[fl_hosp['ProviderName'] == p]['Bed Census'].values[0]/total_staffed_beds
#         elif total_staffed_beds == 0: hosp_occupancy = 0
        else: hosp_occupancy = np.nan
            
        if icu_staffed_beds > 0:
            icu_occupancy = fl_icu[fl_icu['ProviderName'] == p]['Adult ICU Census'].values[0]/icu_staffed_beds
#         elif icu_staffed_beds == 0: icu_occupancy = 0
        else: icu_occupancy = np.nan
        
        override_dict['Staffed All Beds'] = fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        override_dict['Staffed ICU Beds'] = fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        override_dict['All Bed Occupancy Rate'] = hosp_occupancy
        override_dict['ICU Bed Occupancy Rate'] = icu_occupancy
        override_dict['Name'] = p
    
        row_dict = {}

        for col in list(manual_override_cols):
#             print(col)
            # update data for col fields from FL
            if col in list(override_dict.keys()):
                row_dict[col] = override_dict[col]
            # add the other col fields unchanged
            elif len(found_ccm_record[col].values)>0:
                row_dict[col] = found_ccm_record[col].values[0]
            else:
                row_dict[col] = np.nan
        l.append(row_dict)
    else: not_added.append((p,ccm_id))

In [ ]:
 not_added

In [ ]:
new_overrides_df = pd.DataFrame(l)

In [ ]:
new_overrides_df.head()

In [ ]:
manual_override_data = pd.concat([ccm_fl, manual_override_data, new_overrides_df]).drop_duplicates('CCM_ID' , keep='last')

In [ ]:
manual_override_data.iloc[44:80]

In [ ]:
manual_override_data['DH-OBJECTID'] = manual_override_data['DH-OBJECTID'].combine_first(manual_override_data['DH-ID'])
manual_override_data['HCRIS-Provider Number'] = manual_override_data['HCRIS-Provider Number'].combine_first(manual_override_data['HCRIS-ID'])

In [ ]:
manual_override_data.drop(['DH-ID', 'HCRIS-ID'], inplace=True, axis=1)

In [ ]:
# Ensure coordinates are rounded
manual_override_data['Latitude'] = manual_override_data['Latitude'].round(5)
manual_override_data['Longitude'] = manual_override_data['Longitude'].round(5)

In [ ]:
manual_override_data.to_csv(external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'), index=False)